## Load Libraries and Packages

In [1]:
pwd

u'/Users/2015/Dropbox/Data Science/NYC Data Science Academy/class-projects/Rossmann/Scripts/Python'

In [2]:
import datetime
import numpy as np
from numpy import double
import pandas as pd

## Load Data

In [3]:
print("Loading data start...")
train = pd.read_csv("../../input/train.csv")
test = pd.read_csv("../../input/test.csv")
store = pd.read_csv("../../input/store.csv")
states = pd.read_csv("../../input/store_states.csv")
print("Complete!   : )")

Loading data start...
Complete!   : )


/Users/2015/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Process Data:
#### 1: Impute open = 1 for missing open test dasta
#### 2: Separate date column into year, month, day
#### 3: Convert date column to type 'date' and extract:
* day_of_year
* week_of_year
* quarter
* month start
* month end
* quarter start
* quarter end

#### 4: Drop date and standardize StateHoliday
#### 5: Create promotion one feature
#### 6: Merge data
#### 7: Dumify categorical variables

In [4]:
# 1: Impute Open = 1 for store 622 in test data
test['Open'].fillna(1, inplace=True)
print 'Step 1 Complete.'

Step 1 Complete.


In [5]:
# 2: Separate date columns
train['year'] = train.Date.apply(lambda x: x.split('-')[0])
train['year'] = train['year'].astype(int)
train['month'] = train.Date.apply(lambda x: x.split('-')[1])
train['month'] = train['month'].astype(int)
train['day'] = train.Date.apply(lambda x: x.split('-')[2])
train['day'] = train['day'].astype(int)
print 'Step 2 Complete.'

Step 2 Complete.


In [6]:
# 2: Separate date columns
test['year'] = test.Date.apply(lambda x: x.split('-')[0])
test['year'] = test['year'].astype(int)
test['month'] = test.Date.apply(lambda x: x.split('-')[1])
test['month'] = test['month'].astype(int)
test['day'] = test.Date.apply(lambda x: x.split('-')[2])
test['day'] = test['day'].astype(int)
print 'Step 2 Complete.'

Step 2 Complete.


In [7]:
# 3: Withdraw multiple date features
train['Date'] = pd.to_datetime(train['Date'])
train['day_of_year'] = train['Date'].dt.dayofyear
train['quarter'] = train['Date'].dt.quarter
train['is_month_start'] = train['Date'].dt.is_month_start.astype(int)
train['is_month_end'] = train['Date'].dt.is_month_end.astype(int)
train['is_quarter_start'] = train['Date'].dt.is_quarter_start.astype(int)
train['is_quarter_end'] = train['Date'].dt.is_quarter_end.astype(int)
print 'Step 3 Complete.'

Step 3 Complete.


In [8]:
lambdafunc = lambda x: pd.Series([x['Date'].isocalendar()[1]])
train[['weeknum']] = train.apply(lambdafunc, axis=1)

In [9]:
pd.isnull(train).any()

Store               False
DayOfWeek           False
Date                False
Sales               False
Customers           False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
weeknum             False
dtype: bool

In [10]:
train[train.isnull().any(axis=1)]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,weeknum


In [11]:
# 3: Withdraw multiple date features
test['Date'] = pd.to_datetime(test['Date'])
test['day_of_year'] = test['Date'].dt.dayofyear
test['quarter'] = test['Date'].dt.quarter
test['is_month_start'] = test['Date'].dt.is_month_start.astype(int)
test['is_month_end'] = test['Date'].dt.is_month_end.astype(int)
test['is_quarter_start'] = test['Date'].dt.is_quarter_start.astype(int)
test['is_quarter_end'] = test['Date'].dt.is_quarter_end.astype(int)
print 'Step 3 Complete.'

Step 3 Complete.


In [12]:
# datetime.date(2015, 7, 31).isocalendar()[1]
lambdafunc = lambda x: pd.Series([x['Date'].isocalendar()[1]])
test[['weeknum']] = test.apply(lambdafunc, axis=1)

In [13]:
pd.isnull(test).any()

Id                  False
Store               False
DayOfWeek           False
Date                False
Open                False
Promo               False
StateHoliday        False
SchoolHoliday       False
year                False
month               False
day                 False
day_of_year         False
quarter             False
is_month_start      False
is_month_end        False
is_quarter_start    False
is_quarter_end      False
weeknum             False
dtype: bool

In [14]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,year,month,day,day_of_year,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,weeknum
0,1,1,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
1,2,3,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
2,3,7,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
3,4,8,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38
4,5,9,4,2015-09-17,1,1,0,0,2015,9,17,260,3,0,0,0,0,38


In [15]:
# 4: Standardize StateHoliday column.
train.ix[(train['StateHoliday'] == 0), 'StateHoliday'] = '0'
print 'Step 4 Complete.'

Step 4 Complete.


In [16]:
# 4: Standardize StateHoliday column.
test.ix[(test['StateHoliday'] == 0), 'StateHoliday'] = '0'
print 'Step 4 Complete.'

Step 4 Complete.


In [19]:
# 5: Create duration of promo 1
train = train.sort(columns = ['Store', 'Date'], ascending = True, na_position = 'last')
train['PromoFirstDate'] = 0
train.ix[((train['Store'] == train['Store'].shift(1)) & (train['Promo'] == 1 ) & (train['Promo'].shift(1) ==0)), 'PromoFirstDate'] = 1
print 'Step 5 Complete.'

Step 5 Complete.


/Users/2015/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [20]:
# 5: Create duration of promo 1
test = test.sort(columns = ['Store', 'Date'], ascending = True, na_position = 'last')
test['PromoFirstDate'] = 0
test.ix[((test['Store'] == test['Store'].shift(1)) & (test['Promo'] == 1 ) & (test['Promo'].shift(1) ==0)), 'PromoFirstDate'] = 1
print 'Step 5 Complete.'

Step 5 Complete.


/Users/2015/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [21]:
train['PromoSecondDate'] = 0
train.ix[((train['Store'] == train['Store'].shift(2)) & (train['Promo'] == 1 ) & (train['Promo'].shift(2) ==0)), 'PromoSecondDate'] = 1

In [22]:
test['PromoSecondDate'] = 0
test.ix[(test['Store'] == test['Store'].shift(2)) & (test['Promo'] == 1 ) & (test['Promo'].shift(2) ==0), 'PromoSecondDate'] = 1

In [25]:
# check how many stores are listed as Open but have 0 sales and/or 0 customers
train.loc[(train['Open'] == 1) & ((train['Sales'] == 0) | (train['Customers'] == 0))].shape

(0, 21)

In [24]:
# set Open = 0 when Sales = o OR Customers = 0
train.ix[(train['Open'] == 1) & ((train['Sales'] == 0) | (train['Customers'] == 0)), 'Open'] = 0

In [26]:
train['DayBeforeClosed'] = 0
train.ix[((train['Store'] == train['Store'].shift(-1)) & (train['Open'] == 1 ) & (train['Open'].shift(-1) ==0)), 'DayBeforeClosed'] = 1

In [27]:
train['DayAfterClosed'] = 0
train.ix[((train['Store'] == train['Store'].shift(1)) & (train['Open'] == 1 ) & (train['Open'].shift(1) ==0)), 'DayAfterClosed'] = 1

In [29]:
test['DayBeforeClosed'] = 0
test.ix[((test['Store'] == test['Store'].shift(-1)) & (test['Open'] == 1 ) & (test['Open'].shift(-1) ==0)), 'DayBeforeClosed'] = 1

In [31]:
test['DayAfterClosed'] = 0
test.ix[((test['Store'] == test['Store'].shift(1)) & (test['Open'] == 1 ) & (test['Open'].shift(1) ==0)), 'DayAfterClosed'] = 1

In [32]:
train.head(25)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,weeknum,PromoFirstDate,PromoSecondDate,DayBeforeClosed,DayAfterClosed
1016095,1,2,2013-01-01,0,0,0,0,a,1,2013,...,1,1,0,1,0,1,0,0,0,0
1014980,1,3,2013-01-02,5530,668,1,0,0,1,2013,...,1,0,0,0,0,1,0,0,0,1
1013865,1,4,2013-01-03,4327,578,1,0,0,1,2013,...,1,0,0,0,0,1,0,0,0,0
1012750,1,5,2013-01-04,4486,619,1,0,0,1,2013,...,1,0,0,0,0,1,0,0,0,0
1011635,1,6,2013-01-05,4997,635,1,0,0,1,2013,...,1,0,0,0,0,1,0,0,1,0
1010520,1,7,2013-01-06,0,0,0,0,0,1,2013,...,1,0,0,0,0,1,0,0,0,0
1009405,1,1,2013-01-07,7176,785,1,1,0,1,2013,...,1,0,0,0,0,2,1,1,0,1
1008290,1,2,2013-01-08,5580,654,1,1,0,1,2013,...,1,0,0,0,0,2,0,1,0,0
1007175,1,3,2013-01-09,5471,626,1,1,0,1,2013,...,1,0,0,0,0,2,0,0,0,0
1006060,1,4,2013-01-10,4892,615,1,1,0,1,2013,...,1,0,0,0,0,2,0,0,0,0


In [33]:
test.head(25)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,year,month,...,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,weeknum,PromoFirstDate,PromoSecondDate,DayBeforeClosed,DayAfterClosed
40232,40233,1,6,2015-08-01,1,0,0,1,2015,8,...,3,1,0,0,0,31,0,0,1,0
39376,39377,1,7,2015-08-02,0,0,0,1,2015,8,...,3,0,0,0,0,31,0,0,0,0
38520,38521,1,1,2015-08-03,1,1,0,1,2015,8,...,3,0,0,0,0,32,1,1,0,1
37664,37665,1,2,2015-08-04,1,1,0,1,2015,8,...,3,0,0,0,0,32,0,1,0,0
36808,36809,1,3,2015-08-05,1,1,0,1,2015,8,...,3,0,0,0,0,32,0,0,0,0
35952,35953,1,4,2015-08-06,1,1,0,1,2015,8,...,3,0,0,0,0,32,0,0,0,0
35096,35097,1,5,2015-08-07,1,1,0,1,2015,8,...,3,0,0,0,0,32,0,0,0,0
34240,34241,1,6,2015-08-08,1,0,0,1,2015,8,...,3,0,0,0,0,32,0,0,1,0
33384,33385,1,7,2015-08-09,0,0,0,1,2015,8,...,3,0,0,0,0,32,0,0,0,0
32528,32529,1,1,2015-08-10,1,0,0,1,2015,8,...,3,0,0,0,0,33,0,0,0,1


## Store Open on Sunday

In [34]:
train['SundayStore'] = 0
train.ix[(train['Open'] == 1) & (train['DayOfWeek'] == 7), 'SundayStore'] = 1

In [35]:
test['SundayStore'] = 0
test.ix[(test['Open'] == 1) & (test['DayOfWeek'] == 7), 'SundayStore'] = 1

# Join with States Data Frame

In [ ]:
states.head()

In [ ]:
train.shape

In [ ]:
train_states = pd.merge(train, states, on = 'Store', how = 'inner')

In [ ]:
train_states.shape

In [ ]:
train_states.head()

In [ ]:
pd.isnull(train_states).any()

In [ ]:
def rstr(df): return df.shape, df.apply(lambda x: [x.unique()])
print(rstr(train_states))

In [ ]:
print(rstr(test))

In [ ]:
test.shape

In [ ]:
test_states = pd.merge(test, states, on = 'Store', how = 'inner')
print(rstr(test_states))

In [ ]:
pd.isnull(test_states).any()

In [ ]:
test_states.head()

# Output CSV files for R

In [ ]:
pwd

In [ ]:
train_states.to_csv("../../KaggleProject/data/train_states_R_v4.csv", index=False)

In [ ]:
test_states.to_csv("../../KaggleProject/data/test_states_R_v4.csv", index=False)

# Add additional features

In [ ]:
train.loc[(train['Store'] == 1)].groupby(['year', 'month', 'Promo']).mean()

In [ ]:
train.groupby(['Store', 'Promo', 'PromoFirstDate', 'DayOfWeek']).mean()

In [38]:
train.loc[(train['PromoFirstDate'] == 1) & ((train['DayOfWeek'] != 1))].shape

(0, 24)

In [39]:
df_test = train.loc[(train['Store'] == 909) &  (train['year'] == 2015) & (train['month'] >= 6) & (train['month'] <= 8)]

In [49]:
df_test['Closed'] = (1 - df_test['Open']).abs();

/Users/2015/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [51]:
df_test.loc[(df_test['Closed'] == 1) ]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year,...,is_month_end,is_quarter_start,is_quarter_end,weeknum,PromoFirstDate,PromoSecondDate,DayBeforeClosed,DayAfterClosed,SundayStore,Closed
64463,909,4,2015-06-04,0,0,0,1,a,1,2015,...,0,0,0,23,0,0,0,0,0,1
61118,909,7,2015-06-07,0,0,0,0,0,0,2015,...,0,0,0,23,0,0,0,0,0,1
53313,909,7,2015-06-14,0,0,0,0,0,0,2015,...,0,0,0,24,0,0,0,0,0,1
45508,909,7,2015-06-21,0,0,0,0,0,0,2015,...,0,0,0,25,0,0,0,0,0,1
37703,909,7,2015-06-28,0,0,0,0,0,0,2015,...,0,0,0,26,0,0,0,0,0,1
33243,909,4,2015-07-02,0,0,0,1,0,0,2015,...,0,0,0,27,0,0,0,0,0,1
32128,909,5,2015-07-03,0,0,0,1,0,0,2015,...,0,0,0,27,0,0,0,0,0,1
31013,909,6,2015-07-04,0,0,0,0,0,0,2015,...,0,0,0,27,0,0,0,0,0,1
29898,909,7,2015-07-05,0,0,0,0,0,0,2015,...,0,0,0,27,0,0,0,0,0,1
28783,909,1,2015-07-06,0,0,0,0,0,0,2015,...,0,0,0,28,0,0,0,0,0,1
